In [16]:
import requests
import pandas as pd
import os

os.chdir("../")
from glob import glob

In [43]:
df_a = pd.read_excel("https://www.shsu.edu/eco_mwf/Frank_Gini_2018.xls")
# df_a = df_a[["Year", "State", "Gini", ""]]
df_a.columns = [x.lower() for x in df_a.columns]
df_a["state"] = df_a["state"].str.upper()
df_a.head()

,year,st,state,atkin05,gini,rmeandev,theil
0,1917,0,UNITED STATES,0.255703,0.507681,0.792800,0.844601
1,1917,1,ALABAMA,0.156439,0.399792,0.608259,0.446908
2,1917,3,ARIZONA,0.159122,0.399658,0.617671,0.450125
3,1917,4,ARKANSAS,0.127598,0.376483,0.561483,0.324998
4,1917,5,CALIFORNIA,0.192735,0.449363,0.687849,0.553006


In [38]:
glob("data_external/United States/ACSDT5Y*.B19083-*")

['data_external/United States\\ACSDT5Y2010.B19083-2022-11-23T092737.csv',
 'data_external/United States\\ACSDT5Y2011.B19083-2022-11-23T092730.csv',
 'data_external/United States\\ACSDT5Y2012.B19083-2022-11-23T092458.csv',
 'data_external/United States\\ACSDT5Y2013.B19083-2022-11-23T092713.csv',
 'data_external/United States\\ACSDT5Y2014.B19083-2022-11-23T092706.csv',
 'data_external/United States\\ACSDT5Y2015.B19083-2022-11-23T092656.csv',
 'data_external/United States\\ACSDT5Y2016.B19083-2022-11-23T092427.csv',
 'data_external/United States\\ACSDT5Y2017.B19083-2022-11-23T092646.csv',
 'data_external/United States\\ACSDT5Y2018.B19083-2022-11-23T092638.csv',
 'data_external/United States\\ACSDT5Y2019.B19083-2022-11-23T092628.csv',
 'data_external/United States\\ACSDT5Y2020.B19083-2022-11-23T092345.csv']

In [42]:
output = []
for file in glob("data_external/United States/ACSDT5Y*.B19083-*"):
    year = file[35:39]
    df_tmp = pd.read_csv(file)
    df_tmp["year"] = year
    output.append(df_tmp)

df = pd.concat(output, ignore_index=True)
output = []
i = 0
while 3*i < df.shape[0]:
    data = df.iloc[3*i:3*i + 3]

    state = data.iloc[0]["Label (Grouping)"]
    year = data.iloc[0]["year"]
    value = data.iloc[1]["Gini Index"]
    moe = data.iloc[2]["Gini Index"]

    output.append({
        "state": state.upper(),
        "gini": value,
        "moe": moe,
        "year": year
    })
    i += 1

df_b = pd.DataFrame(output)
df_b["year"] = df_b["year"].astype(int)
df_b.head()

df_b = df_a.copy()
df_b.loc[df_b["year"] == 2010, "year"] = 2008
df_b.tail()
df_b["gini"] = df_b["gini"].astype(float)

df_b.to_csv("data_curated/United States/Income_Inequality.csv", index=False)


In [45]:
pd.merge(df_a, df_b, on=["state", "year"])[["gini_x", "gini_y"]].astype(float).corr()

,gini_x,gini_y
gini_x,1.000000,0.998964
gini_y,0.998964,1.000000


In [44]:
df_c = df_a.copy()
df_c.loc[df_c["year"] == 2018, "year"] = 2020
df_c.tail()

df_c.to_csv("data_curated/United States/Income_Inequality.csv", index=False)

In [36]:
df_c.year.unique()

array([1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927,
       1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938,
       1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949,
       1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2020], dtype=int64)

In [30]:
df = pd.concat([df_a[df_a["year"] < 2019], df_b], ignore_index=True)
df["gini"] = df["gini"].astype(float)
df.to_csv("data_curated/United States/Income_Inequality.csv", index=False)